In [ ]:
import pandas as pd
import datetime

In [ ]:
escreen = pd.read_excel(r"RAW DATA.xlsx") #, sheet_name='Invoices')
# stmt = pd.read_csv(r"C:\Users\w501738\Desktop\Python\Escreen\STMT_010820.csv")
insight = pd.read_csv(r"INSIGHT DICTIONARY.csv")

# cleaning the raw file

In [ ]:
escreen_rc = sum(1 for row in escreen['Amount'])
# stmt_rc = sum(1 for row in stmt['CPT'])
insight_rc = sum(1 for row in insight['Visit ID'])


In [ ]:
print(insight_rc)

In [ ]:
i = escreen_rc-1
x = i-1
while i > 0:
    if (pd.isna(escreen['Check No'][i]) ==  True) or (str(escreen['Check No'][i]).lower() == "check no"):
        escreen.drop(escreen.index[i], inplace=True)
        escreen_rc -= 1
        i = x
        x = i-1
    else:
        i-=1
        x = i-1 
if (pd.isna(escreen['Check No'][i]) ==  True) or (str(escreen['Check No'][i]).lower() == "check no"):
        escreen.drop(escreen.index[i], inplace=True)
        escreen_rc -= 1

In [ ]:
escreen.to_excel(r"CLEAN RAW FILE.xlsx")


In [ ]:
escreen1 = pd.read_excel(r"CLEAN RAW FILE.xlsx")
escreen1_rc = sum(1 for row in escreen['Amount'])

# coding and processsing here

In [ ]:
amount = []
coll_date = []
donor_name = []
desc = []

date_and_name_escreen = []

for i in range(escreen1_rc):
    amount.append(escreen1['Amount'][i])
    coll_date.append(escreen1['Collection Date'][i])
    
    name1 = str(escreen1['Donor Name'][i])
    namelist = name1.split(',')
    namelist[-1] = namelist[-1].strip()
    namelist[0] = namelist[0].strip()
    
    namelist[-1] = namelist[-1].lower()
    namelist[0] = namelist[0].lower()
    
    namelist[-1] = namelist[-1].capitalize()
    namelist[0] = namelist[0].capitalize()
    
    newname = namelist[-1]+' '+namelist[0]
    
    donor_name.append(newname)
    
    datename = newname.lower()+' '+str(escreen1['Collection Date'][i])
    date_and_name_escreen.append(datename)
    
    desc.append(escreen1['Description'][i])

In [ ]:
print(date_and_name_escreen[5])

In [ ]:
insight_dict = {}

for i in range(1,insight_rc):
    fname1= insight['Patient First Name'][i]
    fname1 = str(fname1).lower()
    
    lname1= insight['Patient Last Name'][i]
    lname1 = str(lname1).lower()
    
    patname = fname1+" "+lname1
    
    date1 = insight['Days in Visit Date'][i]
    
    date2 = datetime.datetime.strptime(date1, '%m/%d/%y')
#     date3 = date2.strftime('%m/%d/%Y')
#     date4 = date3.date()
    
    date = '{0}/{1}/{2}'.format(date2.month, date2.day, date2.year)
    name_date = patname+' '+str(date)
    
    invoice = int(insight['Invoice Number'][i])
    vid = insight['Visit ID'][i]
    
    if name_date not in insight_dict:
        insight_dict[name_date] = [invoice, vid]
    else:
        l1 = insight_dict[name_date]
        inv = l1[0]
        
        if inv < 0:
            insight_dict[name_date] = [invoice, vid]
        else:
            insight_dict[name_date] = l1

In [ ]:
print(insight_dict['dylan smith 1/1/2020'])

In [ ]:
invoices_col = []
vid = []

for i in range(len(date_and_name_escreen)):
    val = date_and_name_escreen[i]
    if val in insight_dict.keys():
        value = insight_dict[val]
        
        invoice_num = value[0]
        invoices_col.append(value[0])
        
        vid.append(value[-1])
    else:
        invoices_col.append(" ")
        vid.append(" ")

In [ ]:
df = pd.DataFrame(list(zip(amount, coll_date,donor_name, desc, invoices_col, vid)), columns = ['Amount', 'Collection Date', 'Donor Name', 'Description','Invoices', 'Visit ID'])
df.to_excel(r"OUTPUT FILE.xlsx")